In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from os import path as osp
from pandas import DataFrame
from notebook_utils import NotebookUtilities

nu = NotebookUtilities(data_folder_path=osp.abspath('../data'))


# Post-engagement transactions for Affinity Analysis and Association Rule Learning

In [3]:

# Get all CSVs out of one data frame
if nu.pickle_exists('clean_csvs_df'):
    clean_csvs_df = nu.load_object('clean_csvs_df')
    print(clean_csvs_df.shape)
    df = clean_csvs_df.sample(4).dropna(axis='columns', how='all')
    display(df.T)

(149166, 100)


,64530,9203,84532,131851
action_type,TOOL_HOVER,INJURY_RECORD,BAG_ACCESS,TOOL_HOVER
elapsed_time,139446,505843,107099,75094
event_time,2023-05-01 09:48:00,2023-05-23 09:39:00,2023-05-09 08:54:00,2023-04-21 09:42:00
session_uuid,4727e60f-7e08-4595-b565-b6e80b7a8d87,47bb9911-6096-4703-8551-4a0f8cb1063a,8f6097ce-795d-483a-a811-e6a5423bc59d,f68ce2b4-67dc-4829-89fc-cfe478391148
file_name,4727e60f-7e08-4595-b565-b6e80b7a8d87,47bb9911-6096-4703-8551-4a0f8cb1063a,8f6097ce-795d-483a-a811-e6a5423bc59d,max11
time_group,1,2,1,1
injury_record_id,NaN,R Shin Amputation,NaN,NaN
injury_record_patient_id,NaN,Lily_2 Root,NaN,NaN
injury_record_required_procedure,NaN,tourniquet,NaN,NaN
injury_record_severity,NaN,high,NaN,NaN



## Decision Points

<p>
    OSU and Big Bear: With the IRB approval (for the previous OSU dataset) in place, I would like to get working on the dataset asap. That means:
    <ul>
        <li>Getting it deidentified and sharable</li>
        <li>If there is general demographic information like gender or level of expertise that can be associated with the VR performance, that would be useful.</li>
        <li>Big Bear: once it is available to you, I want an exploratory analysis on decision points such that I can see the variance in responses to the same situation.</li>
        <li>For example, if there are 3 waving patients and the participant chooses to move toward one and assess them, I want to know the sim state (visual info. on each patient, distance, etc.) and then a breakdown of how many participants chose to move toward each patient.</li>
        <li>Similar for treatment options, for each patient present in the simulated environment, what is the variance in which treatment is applied</li>
         <li>Does it depend on order?</li>
          <li>Timing?</li>
           <li>Be creative and ask all the questions. The goal is to provide TA1 with this dataset if there is anything useful to be found in analyzing the decisions (not necessarily that we find results but that the data is in shape to ask these questions). Let me know if you need additional clarification.</li>/</p.


### Patient Engagement


These action types mean that the DM has made a decision: INJURY_TREATED, PATIENT_ENGAGED, PULSE_TAKEN, TAG_APPLIED, and TOOL_APPLIED.


### Affinity Analysis and Association Rule Learning using the Apriori Algorithm

In [15]:

def do_affinity_analysis(actions_type, verbose=False):
    if verbose: print(f'{actions_type.title()} Actions for Affinity Analysis and Association Rule Learning', flush=True)
    
    if verbose: print('Collect the inferred rules in a data frame', flush=True)
    if nu.pickle_exists(f'{actions_type}_actions_rules_df'):
        actions_rules_df = nu.load_object(f'{actions_type}_actions_rules_df')
    else:
        if verbose: print('Build the model', flush=True)
        if nu.pickle_exists(f'{actions_type}_actions_itemsets_df'):
            actions_itemsets_df = nu.load_object(f'{actions_type}_actions_itemsets_df')
        else:
            if verbose: print('Convert the list of lists to one hot', flush=True)
            if nu.pickle_exists(f'{actions_type}_actions_one_hot_df'):
                actions_one_hot_df = nu.load_object(f'{actions_type}_actions_one_hot_df')
                print(f'{actions_type}_actions_one_hot_df.shape: {actions_one_hot_df.shape}', flush=True)
            else:
                from mlxtend.preprocessing import TransactionEncoder
                te = TransactionEncoder()
                if verbose: print('Record as actions a more complex example of a patient engagement', flush=True)
                if nu.pickle_exists(f'{actions_type}_actions_list'):
                    actions_list = nu.load_object(f'{actions_type}_actions_list')
                else:
                    actions_list = []
                    if verbose: print('Get all CSVs out of one data frame', flush=True)
                    if nu.pickle_exists('clean_csvs_df'):
                        clean_csvs_df = nu.load_object('clean_csvs_df')
                    print(f'clean_csvs_df.shape: {clean_csvs_df.shape}', flush=True)
                    for (session_uuid, elapsed_time), df in nu.get_session_groupby(mask_series=None, extra_column='elapsed_time'):
                        action_types_list = []
                        if df.shape[0] > 1: action_types_list = df.action_type.tolist()
                        if action_types_list: actions_list.append(action_types_list)
                    nu.store_objects(**{f'{actions_type}_actions_list': actions_list})
                list_length = len(actions_list)
                print(f'We have {list_length:,} simultaneous actions in our list. Here are some examples:', flush=True)
                max_length = 0
                max_list = []
                counter = 0
                import random
                for action_list in actions_list:
                    if (counter < 5) and (random.random() <= 10/list_length):
                        print(action_list)
                        counter += 1
                    if max_length < len(action_list):
                        max_length = len(action_list)
                        max_list = action_list
                print(f'\nIn one sim log, these {max_length} actions occur simultaneously: {nu.conjunctify_nouns(max_list)}.')
                te_ary = te.fit(actions_list).transform(actions_list)
                actions_one_hot_df = DataFrame(te_ary, columns=te.columns_)
                nu.store_objects(**{f'{actions_type}_actions_one_hot_df': actions_one_hot_df})
            print(f'{actions_type}_actions_one_hot_df.shape: {actions_one_hot_df.shape}', flush=True)
            from mlxtend.frequent_patterns import apriori
            actions_itemsets_df = apriori(
                actions_one_hot_df, min_support=0.01, use_colnames=True, max_len=50
            )
            actions_itemsets_df['itemsets_size'] = actions_itemsets_df.itemsets.map(lambda x: len(eval(str(x))))
            nu.store_objects(**{f'{actions_type}_actions_itemsets_df': actions_itemsets_df})
        print(f'{actions_type}_actions_itemsets_df.shape: {actions_itemsets_df.shape}', flush=True)
        display(actions_itemsets_df.sort_values('itemsets_size', ascending=False).head(5))
        from mlxtend.frequent_patterns import association_rules
        actions_rules_df = association_rules(actions_itemsets_df, metric='lift', min_threshold=0.75)
        nu.store_objects(**{f'{actions_type}_actions_rules_df': actions_rules_df})
    print(f'{actions_type}_actions_rules_df.shape: {actions_rules_df.shape}', flush=True)
    display(actions_rules_df.head().T)
    metric_dict = {}
    for metric in [
        'antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction', 'zhangs_metric'
    ]:
        max_metric = actions_rules_df[metric].max()
        print(metric)
        metric_dict[metric] = max_metric
        mask_series = (actions_rules_df[metric] == max_metric)
        display(actions_rules_df[mask_series].head())
    metrics_list = sorted([(metric, max_metric) for metric, max_metric in metric_dict.items()], key=lambda x: x[1], reverse=True)
    display(actions_rules_df.sort_values([x[0] for x in metrics_list], ascending=[False]*len(metrics_list)))
    
    mask_series = (actions_rules_df.antecedents == frozenset({'PATIENT_ENGAGED'}))
    df = actions_rules_df[mask_series].sort_values(['lift', 'antecedent support'], ascending=[False, False])
    if df.head(5).shape[0]:
        print(f'Get the consequent with the highest support for patient engaged')
        display(df.head(5))
    
    mask_series = (actions_rules_df.consequents == frozenset({'PATIENT_ENGAGED'}))
    df = actions_rules_df[mask_series].sort_values(['lift', 'antecedent support'], ascending=[False, False])
    if df.head(5).shape[0]:
        print(f'Get the antecedent with the highest support for patient engaged')
        display(df.head(5))
    
    mask_series = (actions_rules_df.consequents == frozenset({'SESSION_END'}))
    df = actions_rules_df[mask_series].sort_values(['lift', 'antecedent support'], ascending=[False, False])
    if df.head(5).shape[0]:
        print(f'Get the antecedent with the highest support for session end')
        antecedents_list = [str(tuple(x)) for x in df.head(5).antecedents.tolist()]
        if antecedents_list:
            print('The antecedents with highest lift and support to SESSION_END,', end='')
            print(f' and probably therefore to PATIENT_ENGAGEMENT_END, are {nu.conjunctify_nouns(antecedents_list)}.')
        display(df.head(5))
    
    mask_series = (actions_rules_df.antecedents == frozenset({'SESSION_START'}))
    df = actions_rules_df[mask_series].sort_values(['lift', 'antecedent support'], ascending=[False, False])
    if df.head(5).shape[0]:
        print(f'Get the consequent with the highest support for session start')
        display(df.head(5))

In [16]:

do_affinity_analysis(actions_type='next', verbose=False)

next_actions_rules_df.shape: (42, 10)


,0,1,2,3,4
antecedents,(BAG_CLOSED),(TELEPORT),(PATIENT_RECORD),(INJURY_RECORD),(INJURY_RECORD)
consequents,(TELEPORT),(BAG_CLOSED),(INJURY_RECORD),(PATIENT_RECORD),(SESSION_START)
antecedent support,0.129335,0.130035,0.037381,0.035182,0.035182
consequent support,0.130035,0.129335,0.035182,0.037381,0.016192
support,0.129335,0.129335,0.035182,0.035182,0.013993
confidence,1.0,0.99462,0.941176,1.0,0.397727
lift,7.690238,7.690238,26.751337,26.751337,24.563342
leverage,0.112517,0.112517,0.033867,0.033867,0.013423
conviction,inf,161.819248,16.401899,inf,1.633493
zhangs_metric,0.999196,1.0,1.0,0.997721,0.99427


antecedent support


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
17,(VOICE_COMMAND),(S_A_L_T_WAVE_IF_CAN),0.256872,0.053873,0.011694,0.045525,0.845047,-0.002144,0.991254,-0.197913
23,(VOICE_COMMAND),(VOICE_CAPTURE),0.256872,0.255172,0.251974,0.980934,3.844200,0.186427,39.065447,0.995612
41,(VOICE_COMMAND),"(VOICE_CAPTURE, S_A_L_T_WAVE_IF_CAN)",0.256872,0.010195,0.010195,0.039689,3.892996,0.007576,1.030713,1.000000


consequent support


...

zhangs_metric


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(TELEPORT),(BAG_CLOSED),0.130035,0.129335,0.129335,0.994620,7.690238,0.112517,161.819248,1.0
2,(PATIENT_RECORD),(INJURY_RECORD),0.037381,0.035182,0.035182,0.941176,26.751337,0.033867,16.401899,1.0
10,(PATIENT_RECORD),(SESSION_START),0.037381,0.016192,0.016192,0.433155,26.751337,0.015587,1.735586,1.0
12,(S_A_L_T_WALK_IF_CAN),(S_A_L_T_WALKED),0.215292,0.209595,0.209595,0.973538,4.644847,0.164471,29.868981,1.0
15,(S_A_L_T_WAVE_IF_CAN),(S_A_L_T_WAVED),0.053873,0.046677,0.046677,0.866419,18.562152,0.044162,7.136684,1.0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,(INJURY_RECORD),(PATIENT_RECORD),0.035182,0.037381,0.035182,1.000000,26.751337,0.033867,inf,0.997721
11,(SESSION_START),(PATIENT_RECORD),0.016192,0.037381,0.016192,1.000000,26.751337,0.015587,inf,0.978462
26,"(INJURY_RECORD, SESSION_START)",(PATIENT_RECORD),0.013993,0.037381,0.013993,1.000000,26.751337,0.013470,inf,0.976280
14,(S_A_L_T_WAVED),(S_A_L_T_WAVE_IF_CAN),0.046677,0.053873,0.046677,1.000000,18.562152,0.044162,inf,0.992451
0,(BAG_CLOSED),(TELEPORT),0.129335,0.130035,0.129335,1.000000,7.690238,0.112517,inf,0.999196
13,(S_A_L_T_WALKED),(S_A_L_T_WALK_IF_CAN),0.209595,0.215292,0.209595,1.000000,4.644847,0.164471,inf,0.992792
36,"(VOICE_CAPTURE, S_A_L_T_WAVE_IF_CAN)",(VOICE_COMMAND),0.010195,0.256872,0.010195,1.000000,3.892996,0.007576,inf,0.750783
1,(TELEPORT),(BAG_CLOSED),0.130035,0.129335,0.129335,0.994620,7.690238,0.112517,161.819248,1.000000
21,(TOOL_SELECTED),(TOOL_HOVER),0.015392,0.154323,0.015292,0.993506,6.437845,0.012917,130.234283,0.857873
22,(VOICE_CAPTURE),(VOICE_COMMAND),0.255172,0.256872,0.251974,0.987466,3.844200,0.186427,59.287716,0.993341


Get the consequent with the highest support for session start


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
11,(SESSION_START),(PATIENT_RECORD),0.016192,0.037381,0.016192,1.000000,26.751337,0.015587,inf,0.978462
5,(SESSION_START),(INJURY_RECORD),0.016192,0.035182,0.013993,0.864198,24.563342,0.013423,7.104566,0.975077
29,(SESSION_START),"(PATIENT_RECORD, INJURY_RECORD)",0.016192,0.035182,0.013993,0.864198,24.563342,0.013423,7.104566,0.975077



## Do Affinity analysis on the actions that are logged at the exact same time

In [17]:

do_affinity_analysis(actions_type='simultaneous', verbose=False)

simultaneous_actions_rules_df.shape: (42, 10)


,0,1,2,3,4
antecedents,(TELEPORT),(BAG_CLOSED),(PATIENT_RECORD),(INJURY_RECORD),(INJURY_RECORD)
consequents,(BAG_CLOSED),(TELEPORT),(INJURY_RECORD),(PATIENT_RECORD),(SESSION_START)
antecedent support,0.130035,0.129335,0.037381,0.035182,0.035182
consequent support,0.129335,0.130035,0.035182,0.037381,0.016192
support,0.129335,0.129335,0.035182,0.035182,0.013993
confidence,0.99462,1.0,0.941176,1.0,0.397727
lift,7.690238,7.690238,26.751337,26.751337,24.563342
leverage,0.112517,0.112517,0.033867,0.033867,0.013423
conviction,161.819248,inf,16.401899,inf,1.633493
zhangs_metric,1.0,0.999196,1.0,0.997721,0.99427


antecedent support


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
17,(VOICE_COMMAND),(S_A_L_T_WAVE_IF_CAN),0.256872,0.053873,0.011694,0.045525,0.845047,-0.002144,0.991254,-0.197913
23,(VOICE_COMMAND),(VOICE_CAPTURE),0.256872,0.255172,0.251974,0.980934,3.844200,0.186427,39.065447,0.995612
41,(VOICE_COMMAND),"(VOICE_CAPTURE, S_A_L_T_WAVE_IF_CAN)",0.256872,0.010195,0.010195,0.039689,3.892996,0.007576,1.030713,1.000000


consequent support


...

zhangs_metric


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(TELEPORT),(BAG_CLOSED),0.130035,0.129335,0.129335,0.994620,7.690238,0.112517,161.819248,1.0
2,(PATIENT_RECORD),(INJURY_RECORD),0.037381,0.035182,0.035182,0.941176,26.751337,0.033867,16.401899,1.0
10,(PATIENT_RECORD),(SESSION_START),0.037381,0.016192,0.016192,0.433155,26.751337,0.015587,1.735586,1.0
13,(S_A_L_T_WALK_IF_CAN),(S_A_L_T_WALKED),0.215292,0.209595,0.209595,0.973538,4.644847,0.164471,29.868981,1.0
15,(S_A_L_T_WAVE_IF_CAN),(S_A_L_T_WAVED),0.053873,0.046677,0.046677,0.866419,18.562152,0.044162,7.136684,1.0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,(INJURY_RECORD),(PATIENT_RECORD),0.035182,0.037381,0.035182,1.000000,26.751337,0.033867,inf,0.997721
11,(SESSION_START),(PATIENT_RECORD),0.016192,0.037381,0.016192,1.000000,26.751337,0.015587,inf,0.978462
26,"(INJURY_RECORD, SESSION_START)",(PATIENT_RECORD),0.013993,0.037381,0.013993,1.000000,26.751337,0.013470,inf,0.976280
14,(S_A_L_T_WAVED),(S_A_L_T_WAVE_IF_CAN),0.046677,0.053873,0.046677,1.000000,18.562152,0.044162,inf,0.992451
1,(BAG_CLOSED),(TELEPORT),0.129335,0.130035,0.129335,1.000000,7.690238,0.112517,inf,0.999196
12,(S_A_L_T_WALKED),(S_A_L_T_WALK_IF_CAN),0.209595,0.215292,0.209595,1.000000,4.644847,0.164471,inf,0.992792
36,"(VOICE_CAPTURE, S_A_L_T_WAVE_IF_CAN)",(VOICE_COMMAND),0.010195,0.256872,0.010195,1.000000,3.892996,0.007576,inf,0.750783
0,(TELEPORT),(BAG_CLOSED),0.130035,0.129335,0.129335,0.994620,7.690238,0.112517,161.819248,1.000000
20,(TOOL_SELECTED),(TOOL_HOVER),0.015392,0.154323,0.015292,0.993506,6.437845,0.012917,130.234283,0.857873
22,(VOICE_CAPTURE),(VOICE_COMMAND),0.255172,0.256872,0.251974,0.987466,3.844200,0.186427,59.287716,0.993341


Get the consequent with the highest support for session start


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
11,(SESSION_START),(PATIENT_RECORD),0.016192,0.037381,0.016192,1.000000,26.751337,0.015587,inf,0.978462
5,(SESSION_START),(INJURY_RECORD),0.016192,0.035182,0.013993,0.864198,24.563342,0.013423,7.104566,0.975077
29,(SESSION_START),"(PATIENT_RECORD, INJURY_RECORD)",0.016192,0.035182,0.013993,0.864198,24.563342,0.013423,7.104566,0.975077
